In [ ]:
# I used folders in order: 200_1, 200_2, 100, 200_3, 200_4

import import_ipynb
import os
import random
from os import listdir
from os.path import isdir
from numpy import savez_compressed
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from numpy import load
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import perf_counter
import sys
import numpy as np
import io
import json
import math
import matplotlib.image as mpimg
from matplotlib import rcParams
import matplotlib.pyplot as plt
import ast
import tensorflow as tf
from numpy import expand_dims
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from scipy.spatial import distance
import scipy


def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create predication model - note that we have several 
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)

	print(len(results), 'faces have been found')
#--------------------------- detecting the number of faces ------------------------------------
	if len(results) == 0:
		raise Exception('No faces detected')
	elif len(results) > 1:
		raise Exception('Multiple faces detected')		
        
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

def is_match_mass(known_embedding, candidate_embedding, thresh=0.5):
    score = cosine(known_embedding, candidate_embedding)
    return math.floor((1-score)*100)



baseDir = '/Users/denis/Downloads/lfw-deepfunneled200_1' 
outputDir = '/Users/denis/Desktop'
results = 'Person 1, Person 2, Expected, Result, Percent identity \n'

list = []

for entry in os.scandir(baseDir):
    
#     print("entry ----->",entry.path) 
    list.append(entry.path)

index = 0
targetFile = ''

for folder in list:
    targetFileName = os.listdir(folder)[0]
    targetFilePath = folder + '/' + targetFileName
    
    try:
        targetVector = get_embeddings([targetFilePath])
        
        count = 0
        while count < 5 :
            testFolder = random.choice(list)
            if testFolder != folder:
                testFileName = os.listdir(testFolder)[0]
                testFilePath = testFolder + '/' + testFileName
                try:
                    testVector = get_embeddings([testFilePath])
                    result = is_match_mass(targetVector, testVector)
                    textResult = 'Match'
                    print("\n--------------------->", result, index)
                    if result <= 50 :
                        textResult = 'Not Match'
                    print("results---------", results)

                    results = results + targetFileName + ',' + testFileName + ',' + 'Not match' +  ',' + textResult + ',' + str(result) + '\n'

                    count+=1

                except Exception as err:
                    print("---------multiple faces detected...-----------")
                    
    except Exception as err:
        print("---------multiple faces detected-----------")
    
    if index > 5:
        break
    index += 1
    
print(results)
output = open(outputDir + '/results200_test.csv', 'w')
output.write(results)
output.close()


1 faces have been found
1 faces have been found

---------------------> 24 0
results--------- Person 1, Person 2, Expected, Result, Percent identity 

1 faces have been found

---------------------> 30 0
results--------- Person 1, Person 2, Expected, Result, Percent identity 
Deb_Santos_0001.jpg,Daniel_Montgomery_0001.jpg,Not match,Not Match,24

1 faces have been found

---------------------> 22 0
results--------- Person 1, Person 2, Expected, Result, Percent identity 
Deb_Santos_0001.jpg,Daniel_Montgomery_0001.jpg,Not match,Not Match,24
Deb_Santos_0001.jpg,Darrell_Dickey_0001.jpg,Not match,Not Match,30

1 faces have been found

---------------------> 25 0
results--------- Person 1, Person 2, Expected, Result, Percent identity 
Deb_Santos_0001.jpg,Daniel_Montgomery_0001.jpg,Not match,Not Match,24
Deb_Santos_0001.jpg,Darrell_Dickey_0001.jpg,Not match,Not Match,30
Deb_Santos_0001.jpg,David_Stern_0002.jpg,Not match,Not Match,22

1 faces have been found

---------------------> 22 0
results

## 